In [66]:
import pandas as pd
import openpyxl
import gspread
from google.oauth2.service_account import Credentials
from datetime import datetime

In [2]:
# Configuration
XLSX_FILE_PATH = "/Users/arulvasukisrinivasan/PycharmProjects/xlsx/eStatement_Standard Chartered Credit Card_9960_SGD_Nov_2024.xlsx"
CREDENTIALS_FILE = "/Users/arulvasukisrinivasan/PycharmProjects/expenses-tracker-450506-d060c70cd70b.json"
SHEET_NAME = "family_expense_2025_sheet"
NEW_SHEET_TITLE = "Standard_chartered_CSV"

In [4]:
# Authenticate and open Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive"]
credentials = Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=scope)
client = gspread.authorize(credentials)
spreadsheet = client.open(SHEET_NAME)

In [10]:
# Function to process amount and handle 'CR'
def process_amount(value):
    """Convert 'CR' to positive (credited) and others to negative (debited)."""
    if isinstance(value, str) and "CR" in value:
        return abs(float(value.replace(" CR", "").strip()))
    try:
        return -abs(float(value))  # Debit amounts are negative
    except ValueError:
        return 0.0  # Handle errors gracefully

In [5]:
# Check if the worksheet exists, else create it
try:
    sheet = spreadsheet.worksheet(NEW_SHEET_TITLE)
except gspread.exceptions.WorksheetNotFound:
    sheet = spreadsheet.add_worksheet(title=NEW_SHEET_TITLE, rows="100", cols="20")

In [8]:
# Read all sheets from the XLSX file
xls = pd.ExcelFile(XLSX_FILE_PATH, engine='openpyxl')

transactions = []
required_columns = {'Posting Date', 'Description'}  # Set for quick comparison

In [11]:
# Process each sheet
for sheet_name in xls.sheet_names:
    print(f"Processing sheet: {sheet_name}")

    df = pd.read_excel(xls, sheet_name=sheet_name, engine='openpyxl')
    print(f"print excel file {df.columns}")
    # Check if required columns exist
    if not required_columns.issubset(df.columns):
        print(f"Skipping '{sheet_name}' - Missing required columns.")
        continue

    # Process valid rows
    for idx, row in df.iterrows():
        try:
            date = str(row['Posting Date']).strip()
            description = str(row['Description']).strip()
            amount = process_amount(str(row['Amount (SGD)']) if 'Amount (SGD)' in row and pd.notna(row['Amount (SGD)']) else "0")
            remarks = description if description else "credited" if amount > 0 else "debit"
            transactions.append([date, amount, remarks])

        except (ValueError, IndexError) as e:
            print(f"Skipping invalid row {idx + 2} in sheet '{sheet_name}': {e}")

Processing sheet: Table 1
print excel file Index(['ARIVALAGAN KARTHIKEYAN 249 #06-478\nCHOA CHU KANG AVENUE 2\nSINGAPORE 680249',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6',
       'Statement Date                              :   17 Nov 2024\nPayment Due Date                         :   09 Dec 2024\n7020000002479504',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'],
      dtype='object')
Skipping 'Table 1' - Missing required columns.
Processing sheet: Table 2
print excel file Index(['Transaction Date\n04 Nov', 'Posting Date', 'Description', 'Unnamed: 3',
       'Amount (SGD)'],
      dtype='object')
Processing sheet: Table 3
print excel file Index(['Total Relationship Rewards', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Points Earned',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'U

In [97]:
# Process each sheet
for sheet_name in xls.sheet_names:
    print(f"Processing sheet: {sheet_name}")
    if False and not sheet_name == 'Table 2':
        print("skipping {}".format(sheet_name))
        continue

    df = pd.read_excel(xls, sheet_name=sheet_name, engine='openpyxl')
    column_indexs = {
        "Table 1": {
            "transaction_date": 0,
            "description_1": 3,
            "description_2": 10,
            "amount": 14,
        },
        "Table 2": {
            "transaction_date": 0,
            "description_1": 2,
            "description_2": 3,
            "amount": 4,
        }
    }
    transactions = []
    if sheet_name in column_indexs:
        for idx, row in df.iterrows():                
            transaction_date = None
            try:
                transaction_date = row.iloc[column_indexs[sheet_name]['transaction_date']]
                if type(transaction_date) == str:
                    transaction_date = datetime.strptime(transaction_date, "%d %b")  # Expects format like "16 Nov"
            except ValueError:
                pass
            if transaction_date:
                values = []
                values.append(transaction_date)
                values.append(row.iloc[column_indexs[sheet_name]['description_1']])
                values.append(row.iloc[column_indexs[sheet_name]['description_1']])
                values.append(row.iloc[column_indexs[sheet_name]['amount']])
                transactions.append(values)
    print("total transactions - {}".format(len(transactions)))

Processing sheet: Table 1
total transactions - 76
Processing sheet: Table 2
total transactions - 41
Processing sheet: Table 3
total transactions - 0
Processing sheet: Table 4
total transactions - 0


In [98]:
row

Transaction Date\n04 Nov                    NaN
Posting Date                                NaN
Description                 MINIMUM PAYMENT DUE
Unnamed: 3                                  NaN
Amount (SGD)                                 50
Name: 40, dtype: object

In [74]:
row

ARIVALAGAN KARTHIKEYAN 249 #06-478\nCHOA CHU KANG AVENUE 2\nSINGAPORE 680249                                                                                                14 Oct
Unnamed: 1                                                                                                                                                                  18 Oct
Unnamed: 2                                                                                                                                                                     NaN
Unnamed: 3                                                                                                                                          BUS/MRT 519631551 SINGAPORE SG
Unnamed: 4                                                                                                                                                                     NaN
Unnamed: 5                                                                                               

In [55]:
len(row.index)

16

In [57]:
row.items

<bound method Series.items of ARIVALAGAN KARTHIKEYAN 249 #06-478\nCHOA CHU KANG AVENUE 2\nSINGAPORE 680249                                                                            03 Nov
Unnamed: 1                                                                                                                                              04 Nov
Unnamed: 2                                                                                                                                                 NaN
Unnamed: 3                                                                                                                                 51EZPY INCOME 93041
Unnamed: 4                                                                                                                                                 NaN
Unnamed: 5                                                                                                                                                 NaN
Unnamed: 6      

In [60]:
row['Unnamed: 3']

'51EZPY INCOME 93041'

In [61]:
type(row)

pandas.core.series.Series

In [99]:
import pandas as pd
import openpyxl
import gspread
from google.oauth2.service_account import Credentials
from datetime import datetime
import numpy as np

# Configuration
XLSX_FILE_PATH = "/Users/arulvasukisrinivasan/PycharmProjects/xlsx/eStatement_Standard Chartered Credit Card_9960_SGD_Nov_2024.xlsx"
CREDENTIALS_FILE = "/Users/arulvasukisrinivasan/PycharmProjects/expenses-tracker-450506-d060c70cd70b.json"
SHEET_NAME = "family_expense_2025_sheet"
NEW_SHEET_TITLE = "Standard_chartered_CSV"

# Authenticate and open Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive"]
credentials = Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=scope)
client = gspread.authorize(credentials)
spreadsheet = client.open(SHEET_NAME)

# Check if the worksheet exists, else create it
try:
    sheet = spreadsheet.worksheet(NEW_SHEET_TITLE)
except gspread.exceptions.WorksheetNotFound:
    sheet = spreadsheet.add_worksheet(title=NEW_SHEET_TITLE, rows="100", cols="20")

# Function to process amount and handle 'CR'
def process_amount(value):
    """Convert 'CR' to positive (credited) and others to negative (debited)."""
    if isinstance(value, str) and "CR" in value:
        return abs(float(value.replace(" CR", "").strip()))
    try:
        return -abs(float(value))  # Debit amounts are negative
    except ValueError:
        return 0.0  # Handle errors gracefully

# Read all sheets from the XLSX file
xls = pd.ExcelFile(XLSX_FILE_PATH, engine='openpyxl')

transactions = []
required_columns = {'Posting Date', 'Description'}  # Set for quick comparison

# Column indexes for Table 1 and Table 2
column_indexs = {
    "Table 1": {
        "transaction_date": 0,
        "description_1": 3,
        "description_2": 10,
        "amount": 14,
    },
    "Table 2": {
        "transaction_date": 0,
        "description_1": 2,
        "description_2": 3,
        "amount": 4,
    }
}

# Process each sheet
for sheet_name in xls.sheet_names:
    print(f"Processing sheet: {sheet_name}")

    df = pd.read_excel(xls, sheet_name=sheet_name, engine='openpyxl')
    print(f"print excel file {df.columns}")
    # Check if required columns exist
    if not required_columns.issubset(df.columns):
        print(f"Skipping '{sheet_name}' - Missing required columns.")
        continue
        # Ensure Table 1 and Table 2 are being read
    if sheet_name not in column_indexs:
        print(f"Skipping '{sheet_name}' - Not mapped for column indexes.")
        continue

    # Process valid rows
    for idx, row in df.iterrows():
        try:
            transaction_date = None
            # Convert date properly
            if isinstance(transaction_date, str):
                try:
                    transaction_date = datetime.strptime(transaction_date, "%d %b").strftime(
                        '%Y-%m-%d')  # Format like "16 Nov"
                except ValueError:
                    print(f"Invalid date format in row {idx + 2} of {sheet_name}: {transaction_date}")
                    transaction_date = ""
            elif pd.notna(transaction_date):
                transaction_date = pd.to_datetime(transaction_date, errors='coerce').strftime('%Y-%m-%d')
            else:
                transaction_date = ""

            if transaction_date:
                # Extract other columns
                description_1 = str(row.iloc[column_indexs[sheet_name]['description_1']]).strip()
                description_2 = str(row.iloc[column_indexs[sheet_name]['description_2']]).strip()

                # Process amount correctly
                amount_index = column_indexs[sheet_name]['amount']
                amount = process_amount(str(row.iloc[amount_index]) if pd.notna(row.iloc[amount_index]) else "0")

                # Prepare remarks
                remarks = description_1 if description_1 else description_2 if description_2 else "credited" if amount > 0 else "debit"

                # Append transaction
                transactions.append([transaction_date, amount, remarks])
                print(f"Added transaction: {transaction_date}, {amount}, {remarks}")
                print("total transactions - {}".format(len(transactions)))

        except (ValueError, IndexError) as e:
            print(f"Skipping invalid row {idx + 2} in sheet '{sheet_name}': {e}")

# Write headers
headers = [["Date", "Amount", "Remarks"]]
sheet.update(range_name="A1:C1", values=headers)

# Write transactions to Google Sheets
if transactions:
    start_row = 2
    data_range = f"A{start_row}:C{start_row + len(transactions) - 1}"
    sheet.update(range_name=data_range, values=transactions)

    # Calculate totals
    debited_sum = sum(abs(t[1]) for t in transactions if t[1] < 0)
    credited_sum = sum(t[1] for t in transactions if t[1] > 0)
    total_spent = debited_sum

    # Write summary values
    summary_row = len(transactions) + start_row
    summary_values = [
        ["Debited", debited_sum],
        ["Credited Money", credited_sum],
        ["Total Money Spent (This Month)", total_spent]
    ]
    sheet.update(range_name=f"A{summary_row}:B{summary_row + 2}", values=summary_values)

    print("Data successfully written to Google Sheets.")
else:
    print("No valid transactions found.")



Processing sheet: Table 1
print excel file Index(['ARIVALAGAN KARTHIKEYAN 249 #06-478\nCHOA CHU KANG AVENUE 2\nSINGAPORE 680249',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6',
       'Statement Date                              :   17 Nov 2024\nPayment Due Date                         :   09 Dec 2024\n7020000002479504',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'],
      dtype='object')
Skipping 'Table 1' - Missing required columns.
Processing sheet: Table 2
print excel file Index(['Transaction Date\n04 Nov', 'Posting Date', 'Description', 'Unnamed: 3',
       'Amount (SGD)'],
      dtype='object')
Processing sheet: Table 3
print excel file Index(['Total Relationship Rewards', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Points Earned',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'U

In [100]:
# Process each sheet
for sheet_name in xls.sheet_names:
    print(f"Processing sheet: {sheet_name}")

    df = pd.read_excel(xls, sheet_name=sheet_name, engine='openpyxl')
    print(f"print excel file {df.columns}")
    # Check if required columns exist
    if not required_columns.issubset(df.columns):
        print(f"Skipping '{sheet_name}' - Missing required columns.")
        continue
        # Ensure Table 1 and Table 2 are being read
    if sheet_name not in column_indexs:
        print(f"Skipping '{sheet_name}' - Not mapped for column indexes.")
        continue
        

Processing sheet: Table 1
print excel file Index(['ARIVALAGAN KARTHIKEYAN 249 #06-478\nCHOA CHU KANG AVENUE 2\nSINGAPORE 680249',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6',
       'Statement Date                              :   17 Nov 2024\nPayment Due Date                         :   09 Dec 2024\n7020000002479504',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'],
      dtype='object')
Skipping 'Table 1' - Missing required columns.
Processing sheet: Table 2
print excel file Index(['Transaction Date\n04 Nov', 'Posting Date', 'Description', 'Unnamed: 3',
       'Amount (SGD)'],
      dtype='object')
Processing sheet: Table 3
print excel file Index(['Total Relationship Rewards', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Points Earned',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'U